In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [0]:
#import os;os.chdir('/content/drive/My Drive/Colab Notebooks/Lazy courses/NLP2/pretrained_data')
import os;os.chdir('/content/drive/My Drive/Colab Notebooks/Lazy courses/NLP2')
savedir = '/content/drive/My Drive/Colab Notebooks/Lazy courses/NLP2/pretrained_data'

In [0]:
import json
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import expit as sigmoid
from sklearn.utils import shuffle
from datetime import datetime

from scipy.spatial.distance import cosine as cost_dist
from sklearn.metrics.pairwise import pairwise_distances
import sys, os, string


In [0]:
def remove_punctuation(s):
    return s.translate(str.maketrans('','',string.punctuation))

In [0]:
def dot(A, B):
    C = A * B
    val = tf.reduce_sum(C, axis=1)
    val = tf.cast(val, tf.float32)
    return val

In [0]:
def get_wiki():

    V = 20000

    wiki_data_location = '/content/drive/My Drive/Colab Notebooks/Lazy courses/NLP2/wiki_data'
    os.chdir(wiki_data_location)
    
    files = [f for f in os.listdir(wiki_data_location) if f.startswith('enwiki') and f.endswith('txt')]

    #pdb.set_trace()

    #files = glob('enwiki*.txt')
    all_word_counts = {}

    for f in files:
        for line in open(f):
            if line and line[0] not in '[*-|=\{\}':
                s = remove_punctuation(line).lower().split()
                if len(s) > 1:
                    for word in s:
                        if word not in all_word_counts:
                            all_word_counts[word] = 0
                        all_word_counts[word] += 1
    print("Finished counting..")

    V = min(V, len(all_word_counts))
    all_word_counts = sorted(all_word_counts.items(), key=lambda x: x[1], reverse=True)

    top_words = [w for w, count in all_word_counts[:V-1]] + ['<UNK>']
    word2idx = {w:i for i,w in enumerate(top_words)}
    unk = word2idx['<UNK>']

    sents = []
    for f in files:
        for line in open(f):
            if line and line[0] not in '[*-|=\{\}':
                s = remove_punctuation(line).lower().split()
                if len(s) > 1:
                    #if a word is not nearby another word, 
                    #there won't be any context, and hence nothing to train!
                    sent = [word2idx[w] if w in word2idx else unk for w in s]
                    sents.append(sent)

    return sents, word2idx

In [0]:
def get_wiki():

    V = 20000

    wiki_data_location = '/content/drive/My Drive/Colab Notebooks/Lazy courses/NLP2/wiki_data'
    os.chdir(wiki_data_location)
    
    files = [f for f in os.listdir(wiki_data_location) if f.startswith('enwiki') and f.endswith('txt')]

    #pdb.set_trace()

    #files = glob('enwiki*.txt')
    all_word_counts = {}

    for f in files:
        for line in open(f):
            if line and line[0] not in '[*-|=\{\}':
                s = remove_punctuation(line).lower().split()
                if len(s) > 1:
                    for word in s:
                        if word not in all_word_counts:
                            all_word_counts[word] = 0
                        all_word_counts[word] += 1
    print("Finished counting..")

    V = min(V, len(all_word_counts))
    all_word_counts = sorted(all_word_counts.items(), key=lambda x: x[1], reverse=True)

    top_words = [w for w, count in all_word_counts[:V-1]] + ['<UNK>']
    word2idx = {w:i for i,w in enumerate(top_words)}
    unk = word2idx['<UNK>']

    sents = []
    for f in files:
        for line in open(f):
            if line and line[0] not in '[*-|=\{\}':
                s = remove_punctuation(line).lower().split()
                if len(s) > 1:
                    #if a word is not nearby another word, 
                    #there won't be any context, and hence nothing to train!
                    sent = [word2idx[w] if w in word2idx else unk for w in s]
                    sents.append(sent)

    return sents, word2idx

In [0]:
def train_model(savedir):
    #get the data
    sentences, word2idx = get_wiki()

    #debug
    #pdb.set_trace()


    #number of unique words
    vocab_size = len(word2idx)

    #config
    window_size = 5
    learning_rate = 0.025
    final_learning_rate = 0.0001
    num_negatives = 5 # number of negatives samples to draw per input word
    epochs = 20
    D = 50 # word embedding size 

    #learning rate decay
    learning_rate_delta = (learning_rate -final_learning_rate)/epochs

    #params 
    W = np.random.randn(vocab_size, D) #input layer to hidden layer
    V = np.random.randn(D, vocab_size) #hidden to output layer

    #distribution for drawing negative samples
    p_neg = get_negative_sampling_distribution(sentences, vocab_size)

    #create the model
    tf_input = tf.placeholder(tf.int32, shape=(None,))
    tf_negword = tf.placeholder(tf.int32, shape=(None,))
    tf_context = tf.placeholder(tf.int32, shape=(None,)) #targets #context
    tfW = tf.Variable(W)
    tfV = tf.Variable(V.T)

    #correct middle word output
    emb_input = tf.nn.embedding_lookup(tfW, tf_input) # 1 x D
    emb_output = tf.nn.embedding_lookup(tfV, tf_context) # N x D
    correct_output = dot(emb_input, emb_output) # N
    pos_loss = tf.nn.sigmoid_cross_entropy_with_logits(
        labels = tf.ones(tf.shape(correct_output)), logits=correct_output)

    #incorrect middle word output
    emb_input = tf.nn.embedding_lookup(tfW, tf_negword)
    incorrect_output = dot(emb_input, emb_output)
    neg_loss = tf.nn.sigmoid_cross_entropy_with_logits(
        labels=tf.zeros(tf.shape(incorrect_output)), logits = incorrect_output)

    #total loss
    loss = tf.reduce_mean(pos_loss) + tf.reduce_mean(neg_loss) 

    # output = hidden.dot(tfV)

    # loss
    # neither of the built-in TF functions work well
    # per_sample_loss = tf.nn.nce_loss(
    # # per_sample_loss = tf.nn.sampled_softmax_loss(
    #   weights=tfV,
    #   biases=biases,
    #   labels=tfY,
    #   inputs=hidden,
    #   num_sampled=num_negatives,
    #   num_classes=vocab_size,
    # )
    # loss = tf.reduce_mean(per_sample_loss)

    # optimizer
    # train_op = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

    #optimizer
    train_op = tf.train.MomentumOptimizer(0.1, momentum=0.9).minimize(loss)

    #make session
    session = tf.Session()
    init_op = tf.global_variables_initializer()
    session.run(init_op)

    #save the cost to plot them per iteration
    costs = []

    #number of total words in corpus 
    total_words = sum(len(sentence) for sentence in sentences)
    print('Total number of words in corpus:', total_words)

    #for subsampling each sentence
    threshold = 1e-5
    p_drop = 1 - np.sqrt(threshold / p_neg)

    #train the model
    for epoch in range(epochs):
        np.random.shuffle(sentences)

        #accumulate the cost
        cost = 0
        counter = 0
        inputs = []
        targets = []
        negwords = []
        t0 = datetime.now()

        for sentence in sentences:
            #keep only certain words based on p_neg
            sentence = [w for w in sentence if np.random.random() < (1 - p_drop[w])]
            if len(sentence) < 2:
                continue

            #randomly order words so we don't always see samples in same order
            randomly_ordered_positions = np.random.choice(
                len(sentence), size=len(sentence), replace=False)

            for j, pos in enumerate(randomly_ordered_positions):
                #the middle word
                word = sentence[pos]

                #get the positive context words/negative samples
                context_words = get_context(pos, sentence, window_size)
                neg_words = np.random.choice(vocab_size, p=p_neg)

                n = len(context_words)
                inputs += [word]*n
                negwords += [neg_words] * n
                targets += context_words

            if len(inputs) >= 128:
                _,c = session.run((train_op, loss),
                                feed_dict = {
                                    tf_input: inputs,
                                    tf_negword: negwords,
                                    tf_context: targets,
                                })
            
                cost += c

                #reset 
                inputs = [] 
                targets = []
                negwords = []

            counter += 1

            if counter % 100 == 0:
                sys.stdout.write(f"Processed {counter} / {len(sentences)} \n")
                sys.stdout.flush()
            
        # print epoch completion stat
        dt = datetime.now() - t0
        print("epoch complete:", epoch, "cost:", cost, "dt:", dt)

        # save the cost
        costs.append(cost)

        # update the learning rate
        learning_rate -= learning_rate_delta


    # plot the cost per iteration
    plt.plot(costs)
    plt.show()

    #get the params
    W, VT = session.run((tfW, tfV))
    V = VT.T

    #Save the model
    with open(f'{savedir}/word2idx_tf.json', 'w') as f:
        json.dump(word2idx, f)

    np.savez(f"{savedir}/weights_tf.npz", W, V)

    # return the model
    return word2idx, W, V


In [0]:
def get_negative_sampling_distribution(sentences, vocab_size):
    #Pn(w) = probability of word occuring
    #we would like to sample the negative samples
    #such that words that occur more often 
    #should be sampled more often

    word_freq = np.zeros(vocab_size)
    word_count = sum(len(sentence) for sentence in sentences)
    for sentence in sentences:
        for word in sentence:
            word_freq[word] += 1

    #smoothen 
    p_neg = word_freq**0.75

    #normalize
    p_neg = p_neg / p_neg.sum()

    # ####### DEBUG
    # print("###")
    # print(vocab_size)
    # #print(np.all(p_neg > 0))
    # print("###")
    # #######

    assert(np.all(p_neg > 0))
    return p_neg

In [0]:
def get_context(pos, sentence, window_size):
    #input:
    #a sentence of the form: x x x x c c c pos c c c x x x x 
    #output:
    #the context word indices: c c c c c c

    start = max(0, pos - window_size)
    end_ = min(len(sentence), pos + window_size) 

    context = []
    for ctx_pos, ctx_word_idx in enumerate(sentence[start:end_], start=start):
        if ctx_pos != pos:
            #don't include the input word itself as a target
            context.append(ctx_word_idx)
    
    return context

In [0]:
def load_model():
    with open(f"{savedir}/word2idx_tf.json") as f:
        word2idx_tf = json.load(f)
    npz = np.load(f'{savedir}/weights_tf.npz')
    W = npz['arr_0']
    V = npz['arr_1']
    return word2idx_tf, W, V

In [0]:
def analogy(pos1, neg1, pos2, neg2, word2idx, idx2word, W):
    V, D = W.shape

    #pos2 not used in calculation, just printing the expected value
    print(f"testing {pos1} - {neg1} = {pos2} - {neg2}")
    for w in (pos1, neg1, pos2, neg2):
        if w not in word2idx:
            print(f"Sorry {w} is not word2idx")
            return

    p1 = W[word2idx[pos1]]
    n1 = W[word2idx[neg1]]
    p2 = W[word2idx[pos2]]
    n2 = W[word2idx[neg2]]

    vec = p1 - n1 + n2

    distances = pairwise_distances(vec.reshape(1, D), W, metric='cosine').reshape(V)
    idx = distances.argsort()[:10]

    #pick one that's not p1, n1, or n2
    best_idx = -1
    keep_out = [wordidx[w] for w in (pos1, neg1, neg2)]
    #print("Keep_out", keep_out)
    for i in idx:
        if i not in keep_out:
            best_idx = i
            break
    # print("best_idx",best_idx)

    print(f"got: {pos1} - {neg1} = {idx2word[best_idx]} - {neg2}")
    print("closest 10:")
    for i in idx:
        print(idx2word[i], distances[i], '\n')

    print(f"Distance to {pos2}: {cos_dist(p2, vec)}")



In [0]:
def test_model(word2idx, W, V):
  # there are multiple ways to get the "final" word embedding
  # We = (W + V.T) / 2
  # We = W

  idx2word = {i:w for w, i in word2idx.items()}

  for We in (W, (W + V.T) / 2):
    print("**********")

    analogy('king', 'man', 'queen', 'woman', word2idx, idx2word, We)
    analogy('king', 'prince', 'queen', 'princess', word2idx, idx2word, We)
    analogy('miami', 'florida', 'dallas', 'texas', word2idx, idx2word, We)
    analogy('einstein', 'scientist', 'picasso', 'painter', word2idx, idx2word, We)
    analogy('japan', 'sushi', 'england', 'bread', word2idx, idx2word, We)
    analogy('man', 'woman', 'he', 'she', word2idx, idx2word, We)
    analogy('man', 'woman', 'uncle', 'aunt', word2idx, idx2word, We)
    analogy('man', 'woman', 'brother', 'sister', word2idx, idx2word, We)
    analogy('man', 'woman', 'husband', 'wife', word2idx, idx2word, We)
    analogy('man', 'woman', 'actor', 'actress', word2idx, idx2word, We)
    analogy('man', 'woman', 'father', 'mother', word2idx, idx2word, We)
    analogy('heir', 'heiress', 'prince', 'princess', word2idx, idx2word, We)
    analogy('nephew', 'niece', 'uncle', 'aunt', word2idx, idx2word, We)
    analogy('france', 'paris', 'japan', 'tokyo', word2idx, idx2word, We)
    analogy('france', 'paris', 'china', 'beijing', word2idx, idx2word, We)
    analogy('february', 'january', 'december', 'november', word2idx, idx2word, We)
    analogy('france', 'paris', 'germany', 'berlin', word2idx, idx2word, We)
    analogy('week', 'day', 'year', 'month', word2idx, idx2word, We)
    analogy('week', 'day', 'hour', 'minute', word2idx, idx2word, We)
    analogy('france', 'paris', 'italy', 'rome', word2idx, idx2word, We)
    analogy('paris', 'france', 'rome', 'italy', word2idx, idx2word, We)
    analogy('france', 'french', 'england', 'english', word2idx, idx2word, We)
    analogy('japan', 'japanese', 'china', 'chinese', word2idx, idx2word, We)
    analogy('china', 'chinese', 'america', 'american', word2idx, idx2word, We)
    analogy('japan', 'japanese', 'italy', 'italian', word2idx, idx2word, We)
    analogy('japan', 'japanese', 'australia', 'australian', word2idx, idx2word, We)
    analogy('walk', 'walking', 'swim', 'swimming', word2idx, idx2word, We)

In [0]:
if __name__ == '__main__':
    word2idx_tf, W, V = train_model(savedir)
    test_model(word2idx_tf, W, V)

Streaming output truncated to the last 5000 lines.
Processed 449000 / 1271558 
Processed 449100 / 1271558 
Processed 449200 / 1271558 
Processed 449300 / 1271558 
Processed 449400 / 1271558 
Processed 449500 / 1271558 
Processed 449600 / 1271558 
Processed 449700 / 1271558 
Processed 449800 / 1271558 
Processed 449900 / 1271558 
Processed 450000 / 1271558 
Processed 450100 / 1271558 
Processed 450200 / 1271558 
Processed 450300 / 1271558 
Processed 450400 / 1271558 
Processed 450500 / 1271558 
Processed 450600 / 1271558 
Processed 450700 / 1271558 
Processed 450800 / 1271558 
Processed 450900 / 1271558 
Processed 451000 / 1271558 
Processed 451100 / 1271558 
Processed 451200 / 1271558 
Processed 451300 / 1271558 
Processed 451400 / 1271558 
Processed 451500 / 1271558 
Processed 451600 / 1271558 
Processed 451700 / 1271558 
Processed 451800 / 1271558 
Processed 451900 / 1271558 
Processed 452000 / 1271558 
Processed 452100 / 1271558 
Processed 452200 / 1271558 
Processed 452300 / 127155